# Data Warehouse & Star Schema — **Super Simple Notebook**

_Made to be very, very simple. Lots of tiny steps: explanation → code → explanation → code._

_Generated on 2025-09-11._

## 0) What you'll learn (in plain English)
- What a **data warehouse** is, and why it exists.
- What **OLTP vs OLAP** means (and why we separate them).
- What a **star schema** is: one **fact table** in the middle and several **dimension tables** around it.
- How to define the **grain** of a fact table.
- How to build tiny **dimension** and **fact** tables with pandas.
- How to do a simple **star join** and a couple of **analytical queries**.
- A taste of **ETL/ELT** and **Slowly Changing Dimensions (SCD)**, super short and friendly.

## 1) What is a data warehouse?
A **data warehouse (DW)** is a special database used for **analytics** and **reporting**. It collects data from many systems, makes it clean and consistent, and stores it in a shape that is easy to analyze (usually **star schemas**).

Key ideas:
- **Stable** data model meant for reading a lot, not for frequent tiny updates.
- **Subject‑oriented** (e.g., sales, marketing, finance) not app‑feature oriented.
- **Integrated** (different sources become one consistent view).
- **Time‑variant** (we keep history).
- **Non‑volatile** (we load data, we don’t constantly edit old records).

### 1.1) OLTP vs OLAP (super short)
- **OLTP** (Online Transaction Processing): apps that handle day‑to‑day transactions (e.g., shopping cart). Tables are highly normalized, many small writes.
- **OLAP** (Online Analytical Processing): analytics/reporting. We read a lot, summarize, slice/dice. Star schemas help make this fast and simple.

## 2) Dimensional modeling in one minute
- A **fact** is an **event** we can count or sum (e.g., a sale).
- A **dimension** gives **context** about the fact (e.g., product, customer, date).
- A **star schema** = **1 fact** in the middle + **multiple dimensions** around it.
- **Grain**: the exact level of detail in the fact table (e.g., _one row per line item per order_). You must define grain **first**.

> We will build a tiny sales star schema: `FactSales` in the center, with `DimDate`, `DimProduct`, `DimCustomer`.

## 3) Setup (tiny)
We use **pandas** only. No databases. Just DataFrames to show the shape and the joins.

In [8]:
import pandas as pd
pd.set_option('display.width', 120)
pd.set_option('display.max_columns', 20)
print('pandas version:', pd.__version__)

pandas version: 2.3.2


## 4) Define the grain (most important step)
**Grain** of our fact table `FactSales`:
- **One row per**: _sales line item_ (a single product on a single order on a single day).
- Measures: `quantity_sold`, `sales_amount`.
- Foreign keys to dimensions: `date_key`, `product_key`, `customer_key`.

## 5) Build tiny dimension tables
We keep it **super small** and use **surrogate keys** (integers).

In [9]:
DimDate = pd.DataFrame({
    'date_key':   [20240101, 20240102, 20240103],  # yyyymmdd as int for simplicity
    'date':       pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03']),
    'year':       [2024, 2024, 2024],
    'month':      [1, 1, 1],
    'day':        [1, 2, 3],
})
DimProduct = pd.DataFrame({
    'product_key': [1, 2, 3],
    'product_name': ['Widget A', 'Widget B', 'Widget C'],
    'category': ['Widgets', 'Widgets', 'Widgets']
})
DimCustomer = pd.DataFrame({
    'customer_key': [10, 11],
    'customer_name': ['Acme Pty Ltd', 'Bravo Co'],
    'region': ['WA', 'VIC']
})
DimDate, DimProduct, DimCustomer

(   date_key       date  year  month  day
 0  20240101 2024-01-01  2024      1    1
 1  20240102 2024-01-02  2024      1    2
 2  20240103 2024-01-03  2024      1    3,
    product_key product_name category
 0            1     Widget A  Widgets
 1            2     Widget B  Widgets
 2            3     Widget C  Widgets,
    customer_key customer_name region
 0            10  Acme Pty Ltd     WA
 1            11      Bravo Co    VIC)

**Why surrogate keys?**
They make joins simple and stable even if source system IDs change. We **don’t** use natural keys directly in the fact table.

## 6) Build the fact table (matching the grain)
Each row is **one sales line** with the keys pointing to each dimension.

In [10]:
FactSales = pd.DataFrame({
    'date_key':    [20240101, 20240101, 20240102, 20240103],
    'product_key': [1,        2,        2,        3       ],
    'customer_key':[10,       10,       11,       11      ],
    'quantity_sold':[2,       1,        3,        5       ],
    'sales_amount':[200.0,    120.0,    360.0,    750.0   ],
})
FactSales

,date_key,product_key,customer_key,quantity_sold,sales_amount
0,20240101,1,10,2,200.0
1,20240101,2,10,1,120.0
2,20240102,2,11,3,360.0
3,20240103,3,11,5,750.0


## 7) Star join (bring facts + context together)
We join the fact with each dimension using the surrogate keys.

In [13]:
# Join Fact → Dim tables (star join)
star = (FactSales
        .merge(DimDate,   on='date_key',   how='left')
        .merge(DimProduct,on='product_key',how='left')
        .merge(DimCustomer,on='customer_key',how='left')
)
star

,date_key,product_key,customer_key,quantity_sold,sales_amount,date,year,month,day,product_name,category,customer_name,region
0,20240101,1,10,2,200.0,2024-01-01,2024,1,1,Widget A,Widgets,Acme Pty Ltd,WA
1,20240101,2,10,1,120.0,2024-01-01,2024,1,1,Widget B,Widgets,Acme Pty Ltd,WA
2,20240102,2,11,3,360.0,2024-01-02,2024,1,2,Widget B,Widgets,Bravo Co,VIC
3,20240103,3,11,5,750.0,2024-01-03,2024,1,3,Widget C,Widgets,Bravo Co,VIC


In [5]:
star.columns

Index(['date_key', 'product_key', 'customer_key', 'quantity_sold', 'sales_amount', 'date', 'year', 'month', 'day',
       'product_name', 'category', 'customer_name', 'region'],
      dtype='object')

In [27]:
# total_sales
star.groupby(['customer_name','product_name'])['sales_amount'].sum().sort_values(ascending=False)

customer_name  product_name
Bravo Co       Widget C        750.0
               Widget B        360.0
Acme Pty Ltd   Widget A        200.0
               Widget B        120.0
Name: sales_amount, dtype: float64

In [30]:
# total_sales ==> as index = false
star.groupby(['customer_name','product_name'], as_index = False)['sales_amount'].sum().sort_values('product_name')

,customer_name,product_name,sales_amount
0,Acme Pty Ltd,Widget A,200.0
1,Acme Pty Ltd,Widget B,120.0
2,Bravo Co,Widget B,360.0
3,Bravo Co,Widget C,750.0


In [32]:
# mean_sales
star.groupby(by='customer_key')['sales_amount'].mean().sort_values()

customer_key
10    160.0
11    555.0
Name: sales_amount, dtype: float64

In [33]:
# mean_sales ==> reset index
star.groupby(by='customer_key')['sales_amount'].mean().reset_index().sort_values('customer_key', ascending = False)

,customer_key,sales_amount
1,11,555.0
0,10,160.0


In [36]:
print(f"Student's notes:")
print(f"Sort_values() could not be applied to index. If the index column(s) are to be sorted, apply reset_index() or use_index = False.")

Student's notes:
Sort_values() could not be applied to index. If the index column(s) are to be sorted, apply reset_index() or use_index = False.


## 8) Simple analytics (keep it tiny)
### 8.1) Total sales by product
This is a very common type of aggregation in a warehouse.

In [23]:
star.groupby('product_name', as_index=False)['sales_amount'].sum().sort_values('sales_amount', ascending=False)

,product_name,sales_amount
2,Widget C,750.0
1,Widget B,480.0
0,Widget A,200.0


### 8.2) Total quantity by date
Roll up by time is a common pattern (Date dimension is very handy).

In [24]:
star.groupby('date', as_index=False)['quantity_sold'].sum().sort_values('date')

,date,quantity_sold
0,2024-01-01,3
1,2024-01-02,3
2,2024-01-03,5


### 8.3) Total sales by customer and region
Dimensions let you slice results in many ways without changing the fact table.

In [25]:
star.groupby(['customer_name','region'], as_index=False)['sales_amount'].sum().sort_values('sales_amount', ascending=False)

,customer_name,region,sales_amount
1,Bravo Co,VIC,1110.0
0,Acme Pty Ltd,WA,320.0


## 9) Tiny Date Dimension generator (optional but handy)
Warehouses often use a **full Date dimension**. Here’s a **super small** generator.

In [26]:
def build_dim_date(start='2024-01-01', end='2024-01-05'):
    dates = pd.date_range(start, end, freq='D')
    df = pd.DataFrame({'date': dates})
    df['date_key'] = df['date'].dt.strftime('%Y%m%d').astype(int)
    df['year'] = df['date'].dt.year
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    return df[['date_key','date','year','quarter','month','day']]

build_dim_date()

,date_key,date,year,quarter,month,day
0,20240101,2024-01-01,2024,1,1,1
1,20240102,2024-01-02,2024,1,1,2
2,20240103,2024-01-03,2024,1,1,3
3,20240104,2024-01-04,2024,1,1,4
4,20240105,2024-01-05,2024,1,1,5


## 10) ETL/ELT in one breath (the simplest view)
- **Extract** from sources (apps, files, APIs).
- **Transform** (clean, standardize, conform to dimensions, add surrogate keys).
- **Load** into the warehouse (append facts, update dims).

Modern stacks often do **ELT** (load first into a staging area, transform inside the warehouse).

## 11) Slowly Changing Dimensions (SCD) — tiny taste
- **Type 1**: overwrite old attribute (no history). Simple.
- **Type 2**: keep history by adding a new row with a new surrogate key and validity dates. Most common for things like customer attributes that change over time.

We won’t implement a full SCD here, but remember: **facts** don’t change, **dimensions** can.

## 12) Recap — the 5 rules to remember
1. **Pick the grain first** (level of detail per fact row).
2. **Use surrogate keys** for dimensions.
3. **Keep facts numeric and additive/semi‑additive** where possible.
4. **Dimensions give context** (names, categories, dates, regions, etc.).
5. **Star join** makes analytics simple.

That’s it — small pieces, clear structure. You can scale this up later with real ETL and a database.